In [14]:
from transformers import pipeline, AutoTokenizer, BertForTokenClassification


def preprocess(text):
    noktalama_isaretleri = ['!', '?', '.', ',', '-', ':', ';', "'", '[', ']']
    new_text = "".join(
        [char for char in text if char in noktalama_isaretleri or char.isalnum() or char.isspace()])
    new_text_Pure = "".join([char for char in text if char.isalnum(
    ) or char.isspace() or char == "'" or char == "-"])
    new_text_Pure = new_text_Pure.replace("'", " ").replace("-", " ")
    new_text = new_text_Pure.replace("I", "ı").lower()
    print(new_text)
    return new_text


def end2end(sent, capitalization_corr, punc_corr):

    p_sent = preprocess(sent)

    r1 = capitalization_corr(p_sent)
    r2 = punc_corr(p_sent)
    print(r2)

    tokenized_sent = tokenizer.tokenize(p_sent)

    final_sent = ''

    i = 0

    while i < len(tokenized_sent):
        token = tokenized_sent[i]
        if r1[i]['entity'] == 'one':
            token = token.capitalize()
        elif r1[i]['entity'] == 'cap':
            token = token.upper()
            while tokenized_sent[i+1].startswith("##"):
                token += tokenized_sent[i+1][2:].upper()
                i += 1

        if r2[i]['entity'] != 'non':
            token += r2[i]['entity']

        if r2[i]['entity'] not in ["'", "-"]:
            token += ' '

        final_sent += token
        i += 1

    final_sent = final_sent.replace(' ##', '')

    return final_sent


cap_model = BertForTokenClassification.from_pretrained(
    "ytu-ce-cosmos/turkish-base-bert-capitalization-correction")
punc_model = BertForTokenClassification.from_pretrained(
    "ytu-ce-cosmos/turkish-base-bert-punctuation-correction")

tokenizer = AutoTokenizer.from_pretrained(
    "ytu-ce-cosmos/turkish-base-bert-capitalization-correction")


capitalization_corr = pipeline("ner", model=cap_model, tokenizer=tokenizer)
punc_corr = pipeline("ner", model=punc_model, tokenizer=tokenizer)

sent = """
Çünkü benim geldiğim durağı görebilme şansım yok. Konum bazen yetişemiyor, yanlış oluyor, şoföre sürekli sormanız gerekiyor. Halbuki sesli sistem düzgün çalışsa hiçbir problem olmayacak. Veya minibüslerin tabelaları çok küçük, renksiz, ışıksız. Gerçekten anlayamıyoruz.
"""

print(end2end(sent, capitalization_corr, punc_corr))
# Geçen hafta sonu arkadaşlarımla birlikte kısa bir tatile çıktık. Cuma akşamı yola çıktık. Yolculuk oldukça keyifli geçti. Cumartesi sabahı otele vardık. Odalarımıza yerleştikten sonra kahvaltıya indik. Kahvaltıda birçok seçenek vardı; omlet, simit, taze sıkılmış portakal suyu ve çeşitli peynirler. Kahvaltıdan sonra sahile gitmeye karar verdik. Deniz çok sakindi ve hava mükemmeldi. Denizde yüzdük, kumda yürüdük ve güneşlendik. Öğleden sonra şehri gezmeye çıktık. Tarihi yerleri ziyaret ettik ve bol bol fotoğraf çektik. Akşam yemeği için meşhur bir restorana gittik. Deniz ürünleri gerçekten çok tazeydi. Yemek sonrası otele döndüğümüzde çok yorgunduk ama tatilin ilk günü harika geçmişti. Pazar sabahı erken kalkıp bir doğa yürüyüşüne çıktık. Orman içinde yürümek çok huzur vericiydi. Dönüş yolunda biraz trafik vardı ama bu güzel tatilin ardından hiçbiri moralimizi bozamazdı. Eve vardığımızda herkes mutlu ve huzurluydu. Bir sonraki tatili planlamaya başladık bile.


[{'entity': 'non', 'score': 1.0, 'index': 1, 'word': 'Çünkü', 'start': 1, 'end': 6}, {'entity': 'non', 'score': 1.0, 'index': 2, 'word': 'benim', 'start': 7, 'end': 12}, {'entity': 'non', 'score': 1.0, 'index': 3, 'word': 'geldiğim', 'start': 13, 'end': 21}, {'entity': 'non', 'score': 1.0, 'index': 4, 'word': 'durağı', 'start': 22, 'end': 28}, {'entity': 'non', 'score': 1.0, 'index': 5, 'word': 'görebil', 'start': 29, 'end': 36}, {'entity': 'non', 'score': 1.0, 'index': 6, 'word': '##me', 'start': 36, 'end': 38}, {'entity': 'non', 'score': 1.0, 'index': 7, 'word': 'şansım', 'start': 39, 'end': 45}, {'entity': 'non', 'score': 1.0, 'index': 8, 'word': 'yok', 'start': 46, 'end': 49}, {'entity': '.', 'score': 0.9999964, 'index': 9, 'word': '.', 'start': 49, 'end': 50}, {'entity': 'non', 'score': 1.0, 'index': 10, 'word': 'Konum', 'start': 51, 'end': 56}, {'entity': 'non', 'score': 1.0, 'index': 11, 'word': 'bazen', 'start': 57, 'end': 62}, {'entity': 'non', 'score': 1.0, 'index': 12, 'word

In [2]:
sent = """
Dördüncü dersimiz, arkadaşlar ve eğlenceyi hayatından çıkarma
"""

print(end2end(sent, capitalization_corr, punc_corr))

Dördüncü dersimiz arkadaşlar ve eğlenceyi hayatından çıkarma. 
